# Training a new tokenizer from an old one

If a language model is not available in the language you are interested in, or if your corpus is very different from the one your language model was trained on, you will most likely want to retrain the model from scratch using a tokenizer adapted to your data. That will require training a new tokenizer on your dataset. But what exactly does that mean? When we first looked at tokenizers in Chapter 2, we saw that most Transformer models use a `subword tokenization` algorithm. To identify which subwords are of interest and occur most frequently in the corpus at hand, the tokenizer needs to take a hard look at all the texts in the corpus — a process we call training. The exact rules that govern this training depend on the type of tokenizer used, and we’ll go over the three main algorithms later in this chapter.



## 1. Assembling a corpus

There’s a very simple API in 🤗 Transformers that you can use to train a new tokenizer with the same characteristics as an existing one: `AutoTokenizer.train_new_from_iterator()`. To see this in action, let’s say we want to train GPT-2 from scratch, but in a language other than English. Our first task will be to gather lots of data in that language in a training corpus. To provide examples everyone will be able to understand, we won’t use a language like Russian or Chinese here, but rather programming languages.

In [6]:
from datasets import load_dataset

base_url = "https://huggingface.co/datasets/sentence-transformers/codesearchnet/resolve/main/pair/"
data_files = {
    "train": [
        f"{base_url}train-00000-of-00003.parquet",
        f"{base_url}train-00001-of-00003.parquet",
        f"{base_url}train-00002-of-00003.parquet"
    ]
}

raw_datasets = load_dataset("parquet", data_files=data_files)
print(raw_datasets)

pair/train-00000-of-00003.parquet:   0%|          | 0.00/163M [00:00<?, ?B/s]

c:\Users\DELL\.conda\envs\torch_env\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\datasets--sentence-transformers--codesearchnet. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pair/train-00001-of-00003.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

pair/train-00002-of-00003.parquet:   0%|          | 0.00/163M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['comment', 'code'],
        num_rows: 1375067
    })
})


In [14]:
raw_datasets["train"][15]["comment"]

'Creates (or updates) a new ProjectStatus for the given build and\n        returns it.'

Here we will create a new column named `whole_function` to conclude `comment` and `code` column into one concatenated column.

In [15]:
def merge_columns(examples):
    comments = examples["comment"]
    codes = examples["code"]
    whole_function = [
        f"{comment}\n{code}" for comment, code in zip(comments, codes) 
    ]
    return {"whole_function": whole_function}

raw_datasets = raw_datasets.map(merge_columns, batched=True)

Map:   0%|          | 0/1375067 [00:00<?, ? examples/s]

In [17]:
print(raw_datasets["train"][0])

{'comment': 'Computes the new parent id for the node being moved.\n\n@return int', 'code': "protected function parentId()\n\t{\n\t\tswitch ( $this->position )\n\t\t{\n\t\t\tcase 'root':\n\t\t\t\treturn null;\n\n\t\t\tcase 'child':\n\t\t\t\treturn $this->target->getKey();\n\n\t\t\tdefault:\n\t\t\t\treturn $this->target->getParentId();\n\t\t}\n\t}", 'whole_function': "Computes the new parent id for the node being moved.\n\n@return int\nprotected function parentId()\n\t{\n\t\tswitch ( $this->position )\n\t\t{\n\t\t\tcase 'root':\n\t\t\t\treturn null;\n\n\t\t\tcase 'child':\n\t\t\t\treturn $this->target->getKey();\n\n\t\t\tdefault:\n\t\t\t\treturn $this->target->getParentId();\n\t\t}\n\t}"}


The first thing we need to do is transform the dataset into an `iterator` of lists of texts — for instance, a list of list of texts. Using lists of texts will enable our tokenizer to go faster (training on batches of texts instead of processing individual texts one by one), and it should be an iterator if we want to avoid having everything in memory at once. If your corpus is huge, you will want to take advantage of the fact that 🤗 Datasets does not load everything into RAM but stores the elements of the dataset on disk.



Using a Python generator, we can avoid Python loading anything into memory until it’s actually necessary. To create such a generator, you just to need to replace the brackets with parentheses:



In [18]:
training_corpus = (
    raw_datasets["train"][i: i + 1000]["whole_function"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

This line of code doesn’t fetch any elements of the dataset; it just creates an object you can use in a Python `for` loop. The texts will only be loaded when you need them (that is, when you’re at the step of the `for` loop that requires them), and only 1,000 texts at a time will be loaded. This way you won’t exhaust all your memory even if you are processing a huge dataset.



The problem with a `generator` object is that it can only be used once. So, instead of this giving us the list of the first 10 digits twice:



In [19]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


That’s why we define a function that returns a `generator` instead:



In [20]:
def get_training_corpus():
    return (
        raw_datasets["train"][i: i + 1000]["whole_function"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )

You can also define your generator inside a `for` loop by using the `yield` statement:



In [ ]:
def get_training_corpus():
    datasets = raw_datasets["train"]
    for start_idx in range(0, len(datasets), 1000):
        samples = datasets[start_idx: start_idx + 1000]
        yield samples["whole_function"]

which will produce the exact same generator as before, but allows you to use more complex logic than you can in a list comprehension.



## 2. Training a new tokenizer

Now that we have our corpus in the form of an iterator of batches of texts, we are ready to train a new tokenizer. To do this, we first need to load the tokenizer we want to pair with our model (here, GPT-2):



In [22]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch. This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; our new tokenizer will be exactly the same as GPT-2, and the only thing that will change is the vocabulary, which will be determined by the training on our corpus.



First let’s have a look at how this tokenizer would treat an example function:



In [23]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

This tokenizer has a few special symbols, like `Ġ` and `Ċ`, which denote spaces and newlines, respectively. As we can see, this is not too efficient: the tokenizer returns individual tokens for each space, when it could group together indentation levels (since having sets of four or eight spaces is going to be very common in code). It also split the function name a bit weirdly, not being used to seeing words with the `_` character.



Let’s train a new tokenizer and see if it solves those issues. For this, we’ll use the method `train_new_from_iterator()`:



In [24]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

Let’s try our brand new tokenizer on the previous example:

In [25]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`.',
 '"""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [27]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

28
36


Let’s look at another example:



In [28]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

['class',
 'ĠLinear',
 'Layer',
 '():',
 'ĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'init',
 '__(',
 'self',
 ',',
 'Ġinput',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'weight',
 'Ġ=',
 'Ġtorch',
 '.',
 'rand',
 'n',
 '(',
 'input',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'bias',
 'Ġ=',
 'Ġtorch',
 '.',
 'zeros',
 '(',
 'output',
 '_',
 'size',
 ')',
 'ĊĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'call',
 '__(',
 'self',
 ',',
 'Ġx',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġx',
 'Ġ@',
 'Ġself',
 '.',
 'weights',
 'Ġ+',
 'Ġself',
 '.',
 'bias',
 'ĊĠĠĠĠ']

In addition to the token corresponding to an indentation, here we can also see a token for a double indentation: `ĊĠĠĠĠĠĠĠ`. The special Python words like `class`, `init`, `call`, `self`, and `return` are each tokenized as one token, and we can see that as well as splitting on `_` and `.` the tokenizer correctly splits even camel-cased names: `LinearLayer` is tokenized as `["ĠLinear", "Layer"]`.



## 3. Saving the tokenizer


To make sure we can use it later, we need to save our new tokenizer. Like for models, this is done with the `save_pretrained()` method:



In [33]:
tokenizer.save_pretrained("code-search-net-tokenizer")

('code-search-net-tokenizer\\tokenizer_config.json',
 'code-search-net-tokenizer\\tokenizer.json')

This will create a new folder named `code-search-net-tokenizer`, which will contain all the files the tokenizer needs to be reloaded. If you want to share this tokenizer with your colleagues and friends, you can upload it to the Hub by logging into your account. If you’re working in a notebook, there’s a convenience function to help you with this:



In [29]:
from huggingface_hub import notebook_login

notebook_login()

Once you’ve logged in, you can push your tokenizer by executing the following command:



In [30]:
tokenizer.push_to_hub("code-search-net-tokenizer")

CommitInfo(commit_url='https://huggingface.co/arraypowerplay/code-search-net-tokenizer/commit/fa5df0981353959138d8da6e774d7300c60c5496', commit_message='Upload tokenizer', commit_description='', oid='fa5df0981353959138d8da6e774d7300c60c5496', pr_url=None, repo_url=RepoUrl('https://huggingface.co/arraypowerplay/code-search-net-tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='arraypowerplay/code-search-net-tokenizer'), pr_revision=None, pr_num=None)

This will create a new repository in your namespace with the name `code-search-net-tokenizer`, containing the tokenizer file. You can then load the tokenizer from anywhere with the `from_pretrained()` method:



In [32]:
tokenizer = AutoTokenizer.from_pretrained("arraypowerplay/code-search-net-tokenizer")

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

c:\Users\DELL\.conda\envs\torch_env\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--arraypowerplay--code-search-net-tokenizer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json: 0.00B [00:00, ?B/s]